In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the YouTube channel videos page
url = "https://www.youtube.com/@PW-Foundation/videos"

# Send a GET request to fetch the page content
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first five video containers
video_containers = soup.find_all('a', {'id': 'video-title'}, limit=5)

# Extract the required data
video_urls = [f"https://www.youtube.com{video['href']}" for video in video_containers]
titles = [video['title'] for video in video_containers]

# Find the thumbnails, views, and posted times (may not be as straightforward)
thumbnail_urls = []
view_counts = []
post_times = []

for video in video_containers:
    parent = video.find_parent('ytd-grid-video-renderer')
    thumbnail_tag = parent.find('img', {'id': 'img'})
    if thumbnail_tag:
        thumbnail_urls.append(thumbnail_tag['src'])
    else:
        thumbnail_urls.append('N/A')
    
    metadata = parent.find_all('span', {'class': 'style-scope ytd-grid-video-renderer'})
    if metadata and len(metadata) >= 2:
        view_counts.append(metadata[0].text)
        post_times.append(metadata[1].text)
    else:
        view_counts.append('N/A')
        post_times.append('N/A')

# Save data to a CSV file
data = {
    "Video URL": video_urls,
    "Thumbnail URL": thumbnail_urls,
    "Title": titles,
    "Views": view_counts,
    "Posted Time": post_times
}

df = pd.DataFrame(data)
df.to_csv('youtube_videos.csv', index=False)

print("Data has been successfully scraped and saved to youtube_videos.csv")


Data has been successfully scraped and saved to youtube_videos.csv
